In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint,uniform

# General Parameters

In [2]:
labels = [0,1,2,3,4]

n_pop = 10000
n_loc = 2601
n_net = 250

person_attr = 4  # 4 attributes for a person. 


transmit_rate = 0.2
e_to_i_rate = 0.2
recovery_rate = 0.1
death_fraction= 0.01
background_infection_rate = 0.01

random_test_rate =0.1
test_sensitivity =0.99
finish_test_rate =1.0

# Step 1: Create a population

Individuals are defined by `['state', 'home_location', 'network_location', 'current_location' ]`, for example: 

`[0, 0, 4, 0]` is an individual who is susceptible, whose home is `0` and whose work is `4`, and who is currently at home.

In [3]:
def create_person(state, home):
    net = randint(0,n_net)
    return np.array([state, home , net , home ])    # Create a person in state 'state', initially at home


pop = np.empty((n_pop,person_attr),int)          # Empty population

link_weight = np.zeros(n_pop,float)


for i in range(0,10):                         # Set one infected person  with random network
    pop[i] = create_person(2,i)
    link_weight[i] = 0.1

for i in range(10,n_pop):
    home = randint(0,n_loc)                      # Assign random homes for remainder with random network
    pop[i] = create_person(0,home)
    link_weight[i] = 0.1


In [ ]:
#def Gillespie_Run(tf):

output = np.zeros(6,int)

n_sus = len(np.where(pop[:,0]==0)[0])
n_exp = len(np.where(pop[:,0]==1)[0])
n_inf = len(np.where(pop[:,0]==2)[0])
n_rec = len(np.where(pop[:,0]==3)[0])
n_dead= len(np.where(pop[:,0]==4)[0])


t = 0.0

t_out = 0.0

output[0] = t
output[1] = n_sus
output[2] = n_exp
output[3] = n_inf
output[4] = n_rec
output[5] = n_dead

is_confined = [False]*n_pop
being_tested = [False]*n_pop

counter = 0
while(n_exp+n_inf>0):

    if(t>=t_out):
        output = np.vstack((output,[t_out,n_sus,n_exp,n_inf,n_rec,n_dead]))
        t_out += 1
        print(t)
        
    a0 = 0
    
    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==0):                  # If they are susceptible S-> E
            n_infected_in_location = len(np.where(pop[np.where(pop[:,3]==pop[i][3])][:,0]==2)[0])
            a0 += transmit_rate * n_infected_in_location + background_infection_rate
    a1 = a0
 
    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==1):                  # If they are exposed E-> I
            a0 += e_to_i_rate
    a2 = a0
    
    for i in range(0,n_pop):               # For every person
        if(pop[i][0]==2):                  # If they are infected I -> R // D
            a0 += recovery_rate
    a3 = a0
    
    for i in range(0,n_pop):               # For every person
        if(not is_confined[i] and pop[i][0]!=4): # If they aren't confined or dead
            a0 += random_test_rate
    a4 = a0
    
    for i in range(0,n_pop):               # For every person
        if(being_tested[i]):               # If they are being tested
            a0 += finish_test_rate
    a5 = a0
    
    # Shift people in network
    
    for i in range(0,n_pop):                # For every person
        person = pop[i]
        
        if (not is_confined[i] and person[0]!=4): # If they aren't confined or dead
            a0 += link_weight[i]                  # move them to their other location
    
    
    dt = -np.log(uniform(0.0, 1.0)) / a0
    t = t + dt  
    
    a = uniform(0.0,1.0) * a0                     # Choose which process occurs
    
    k = 0
    
#     print("a",str(a))
#     print("a1",str(a1))
#     print("a2",str(a2))
    
    if(a<a1):  # In this case, an S-> E
#         print("S->E")
        psum = 0
        for i in range(0,n_pop):
            if pop[i][0] == 0:
                n_infected_in_location = len(np.where(pop[np.where(pop[:,3]==pop[i][3])][:,0]==2)[0])
                psum += transmit_rate * n_infected_in_location + background_infection_rate
                
                if (psum>a):
                    k=i
                    break
            
        pop[k][0]=1
            
    elif(a<a2): # Here E-> I
#         print("E->I")
        psum = a1
        
        for i in range(0,n_pop):
            if(pop[i][0]==1):
                psum += e_to_i_rate
                
                if (psum>a):
                    k=i
                    break;
            
        pop[k][0]=2
        
    elif(a<a3): # I -> R || D
#         print("I->RD")
        psum = a2

        for i in range(0,n_pop):

            if(pop[i][0]==2):
                psum += recovery_rate

                if(psum>a):
                    k=i
                    break
                
        is_confined[k] = False


        if(uniform(0.0,1.0)<death_fraction):  # Testing if person died or recovered
            pop[k][0]=4  # Oh no! Dead!
        else:
            pop[k][0]=3  # Yay! Recovered!
                
        
    elif(a<a4):
        psum = a3
#         print("Random test")
        for i in range(0,n_pop):
            if(not is_confined[i] and pop[i][0]!=4):  # If the person isn't confined or dead
                psum += random_test_rate
                if(psum>a):
                    k=i
                    break
                
        being_tested[k]=True
        
    elif(a<a5):
        psum = a4
#         print("test result")
        for i in range(0,n_pop):
                
            if being_tested[i] == True:
                psum += finish_test_rate

                if(psum>a):
                    k=i
                    break
            
        being_tested[k]=False    

        if (pop[k][0]==2 and uniform(0.0,1.0)<test_sensitivity): # If the person is infected and the test works
            is_confined[k] = True                                # They are confined
            
    else:  # All else failing, move people around
        
        psum = a5
#         print("shift")
        for i in range(0,n_pop):
            person = pop[i]
        
            if (not is_confined[i] and person[0]!=4): # If they aren't confined or dead
                psum += link_weight[i]                  # move them to their other location
                
                if(psum>a):
                    k=i
                    break
                
        if(pop[i][3]==pop[i][1]):                   # If the person is at home
            pop[i][3] = pop[i][2]                   # move them to their network location
        else:
            pop[i][3] = pop[i][1]                   # otherwise move them home
    
    
    n_sus = len(np.where(pop[:,0]==0)[0])
    n_exp = len(np.where(pop[:,0]==1)[0])
    n_inf = len(np.where(pop[:,0]==2)[0])
    n_rec = len(np.where(pop[:,0]==3)[0])
    n_dead= len(np.where(pop[:,0]==4)[0])
    
    
                                           # WHILE Loop ended

0.0
1.0003693039033739
2.0005319038479774
3.0000694368879994
4.000309174265444
5.000509677726878
6.000031019436708
7.000159769894091
8.00021606204545
9.000313535533435
10.000099979154307


In [ ]:
np.where(pop[np.where(pop[:,3]==pop[0][3])][:,0]==2)[0]

In [ ]:
print(output)

In [ ]:
plt.plot(output[:,0], output[:,1])
plt.show()
plt.plot(output[:,0], output[:,2])
plt.show()
plt.plot(output[:,0], output[:,3])
plt.show()
plt.plot(output[:,0], output[:,4])
plt.show()
plt.plot(output[:,0], output[:,5])
plt.show()

In [ ]:
np.savetxt('sample_data_Philip_run',output, delimiter=' ')